In [93]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

In [24]:
df = pd.read_csv('./Data/Hotel_Reservations.csv')

In [25]:
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [27]:
df.describe()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
count,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000
mean,1.844962,0.105279,0.810724,2.204300,0.030986,85.232557,2017.820427,7.423653,15.596995,0.025637,0.023349,0.153411,103.423539,0.619655
std,0.518715,0.402648,0.870644,1.410905,0.173281,85.930817,0.383836,3.069894,8.740447,0.158053,0.368331,1.754171,35.089424,0.786236
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2017.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,17.000000,2018.000000,5.000000,8.000000,0.000000,0.000000,0.000000,80.300000,0.000000
50%,2.000000,0.000000,1.000000,2.000000,0.000000,57.000000,2018.000000,8.000000,16.000000,0.000000,0.000000,0.000000,99.450000,0.000000
75%,2.000000,0.000000,2.000000,3.000000,0.000000,126.000000,2018.000000,10.000000,23.000000,0.000000,0.000000,0.000000,120.000000,1.000000
max,4.000000,10.000000,7.000000,17.000000,1.000000,443.000000,2018.000000,12.000000,31.000000,1.000000,13.000000,58.000000,540.000000,5.000000


In [28]:
df['arrival_date'] = df['arrival_year'].astype(str) + '-' + df['arrival_month'].astype(str) + '-' + df['arrival_date'].astype(str)
df.loc[:, 'arrival_date'] = pd.to_datetime(df['arrival_date'], format='%Y-%m-%d', errors='coerce')
df = df.dropna(subset=['arrival_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36238 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Booking_ID                            36238 non-null  object        
 1   no_of_adults                          36238 non-null  int64         
 2   no_of_children                        36238 non-null  int64         
 3   no_of_weekend_nights                  36238 non-null  int64         
 4   no_of_week_nights                     36238 non-null  int64         
 5   type_of_meal_plan                     36238 non-null  object        
 6   required_car_parking_space            36238 non-null  int64         
 7   room_type_reserved                    36238 non-null  object        
 8   lead_time                             36238 non-null  int64         
 9   arrival_year                          36238 non-null  int64         
 10

In [29]:
df['timedelta'] = pd.to_timedelta(df['lead_time'], unit='D')
df['booking_date'] = df['arrival_date'] - df['timedelta']
df = df.drop(columns=['timedelta'])
df['booking_month'] = df['booking_date'].dt.month

In [30]:
df = df.drop(columns = ['Booking_ID', 'arrival_year'], axis = 1)

In [31]:
df['no_of_people'] = df['no_of_adults'] + df['no_of_children']
df['no_of_nights'] = df['no_of_weekend_nights'] + df['no_of_week_nights']
df['no_of_prior_bookings'] = df['no_of_previous_cancellations'] + df['no_of_previous_bookings_not_canceled']

In [32]:
df.describe()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_month,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_month,no_of_people,no_of_nights,no_of_prior_bookings
count,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000,36238.000000
mean,1.845301,0.105221,0.810475,2.204206,0.030934,85.275070,7.429190,0.025553,0.023346,0.152961,103.437259,0.619957,6.109195,1.950522,3.014681,0.176307
std,0.518572,0.402540,0.870992,1.410784,0.173142,85.953561,3.066563,0.157801,0.368483,1.753366,35.084264,0.786403,3.571383,0.650046,1.786058,1.953375
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,17.000000,5.000000,0.000000,0.000000,0.000000,80.300000,0.000000,3.000000,2.000000,2.000000,0.000000
50%,2.000000,0.000000,1.000000,2.000000,0.000000,57.000000,8.000000,0.000000,0.000000,0.000000,99.450000,0.000000,6.000000,2.000000,3.000000,0.000000
75%,2.000000,0.000000,2.000000,3.000000,0.000000,126.000000,10.000000,0.000000,0.000000,0.000000,120.000000,1.000000,9.000000,2.000000,4.000000,0.000000
max,4.000000,10.000000,7.000000,17.000000,1.000000,443.000000,12.000000,1.000000,13.000000,58.000000,540.000000,5.000000,12.000000,12.000000,24.000000,64.000000


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36238 entries, 0 to 36274
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   no_of_adults                          36238 non-null  int64         
 1   no_of_children                        36238 non-null  int64         
 2   no_of_weekend_nights                  36238 non-null  int64         
 3   no_of_week_nights                     36238 non-null  int64         
 4   type_of_meal_plan                     36238 non-null  object        
 5   required_car_parking_space            36238 non-null  int64         
 6   room_type_reserved                    36238 non-null  object        
 7   lead_time                             36238 non-null  int64         
 8   arrival_month                         36238 non-null  int64         
 9   arrival_date                          36238 non-null  datetime64[ns]
 10

In [34]:
df['booking_status'].value_counts()

Not_Canceled    24360
Canceled        11878
Name: booking_status, dtype: int64

In [37]:
df['booking_year'] = df['booking_date'].dt.year
df['booking_day'] = df['booking_date'].dt.day
df['arrival_year'] = df['arrival_date'].dt.year
df['arrival_day'] = df['arrival_date'].dt.day
reordered = ['booking_status', 'no_of_people', 'no_of_adults', 'no_of_children', 'no_of_nights', 'no_of_weekend_nights', 'no_of_week_nights', 'booking_year', 'booking_month', 'booking_day', 'lead_time', 'arrival_year', 'arrival_month', 'arrival_day', 'type_of_meal_plan', 'market_segment_type', 'room_type_reserved', 'avg_price_per_room', 'required_car_parking_space', 'repeated_guest', 'no_of_prior_bookings', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 'no_of_special_requests']
df_cleaned = df[reordered]
df_cleaned.loc[:, 'type_of_meal_plan'] = df_cleaned['type_of_meal_plan'].replace({'Not Selected': 0, 'Meal Plan 1': 1, 'Meal Plan 2': 2, 'Meal Plan 3': 3})
#df_cleaned.loc[:, 'room_type_reserved'] = df_cleaned['room_type_reserved'].replace({'Room_Type 1': 1, 'Room_Type 2': 2, 'Room_Type 3': 3, 'Room_Type 4': 4, 'Room_Type 5': 5, 'Room_Type 6': 6, 'Room_Type 7': 7})
df_cleaned.loc[:, 'market_segment_type'] = df_cleaned['market_segment_type'].replace({'Online': 0, 'Offline': 1, 'Corporate': 2, 'Complementary': 2, 'Aviation': 2})
df_cleaned.loc[:, 'booking_status'] = df_cleaned['booking_status'].replace({'Not_Canceled': 0, 'Canceled': 1})


In [38]:
df_cleaned['room_type_reserved'].value_counts()

Room_Type 1    28105
Room_Type 4     6049
Room_Type 6      964
Room_Type 2      692
Room_Type 5      263
Room_Type 7      158
Room_Type 3        7
Name: room_type_reserved, dtype: int64

In [51]:
X = df_cleaned.drop(columns = 'booking_status', axis = 1)
y = df_cleaned['booking_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 100)

In [52]:
#ord_subset_train = X_train['room_type_reserved']
#ord_subset_test = X_test['room_type_reserved']

#room_type = ['Room_Type 3', 'Room_Type 2', 'Room_Type 1', 'Room_Type 5', 'Room_Type 4', 'Room_Type 7', 'Room_Type 6']

#X_train_room_type = ord_subset_train.values.reshape(-1, 1)
#X_test_room_type = ord_subset_test.values.reshape(-1, 1)

#o_enc = OrdinalEncoder(categories = [room_type])
#X_train_room_type_encoded = pd.Series(o_enc.fit_transform(X_train_room_type), columns = ord_subset_train.columns)
#X_test_room_type_encoded = pd.DataFrame(o_enc.transform(X_test_room_type))
#X_train_room_type_encoded

In [73]:
ohe = OneHotEncoder(drop="first", sparse=False)
X_train_nominal = X_train[['booking_year', 'booking_month', 'booking_day', 'arrival_year', 'arrival_month', 'arrival_day', 'type_of_meal_plan', 'market_segment_type', 'required_car_parking_space', 'repeated_guest', 'room_type_reserved']]
X_test_nominal = X_test[['booking_year', 'booking_month', 'booking_day', 'arrival_year', 'arrival_month', 'arrival_day', 'type_of_meal_plan', 'market_segment_type', 'required_car_parking_space', 'repeated_guest', 'room_type_reserved']]
y_train_reshaped = np.array(y_train).reshape(-1, 1)
y_test_reshaped = np.array(y_test).reshape(-1, 1)

X_train_nominal_encoded = pd.DataFrame(ohe.fit_transform(X_train_nominal))
X_test_nominal_encoded = pd.DataFrame(ohe.transform(X_test_nominal))
y_train = pd.DataFrame(ohe.fit_transform(y_train_reshaped))
y_test = pd.DataFrame(ohe.transform(y_test_reshaped))

In [76]:
scaler = StandardScaler()
X_train_num = X_train[['no_of_people', 'no_of_adults', 'no_of_children', 'no_of_nights', 'no_of_weekend_nights', 'no_of_week_nights', 'lead_time', 'avg_price_per_room', 'no_of_prior_bookings', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 'no_of_special_requests']]
X_test_num = X_test[['no_of_people', 'no_of_adults', 'no_of_children', 'no_of_nights', 'no_of_weekend_nights', 'no_of_week_nights', 'lead_time', 'avg_price_per_room', 'no_of_prior_bookings', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 'no_of_special_requests']]

X_train_num_scaled = pd.DataFrame(scaler.fit_transform(X_train_num))
X_test_num_scaled = pd.DataFrame(scaler.transform(X_test_num))

In [80]:
X_train = pd.concat([X_train_nominal_encoded, X_train_num_scaled], axis = 1)
X_test = pd.concat([X_test_nominal_encoded, X_test_num_scaled], axis = 1)

In [91]:
logreg = LogisticRegression(max_iter = 1000, random_state = 100)
logreg.fit(X_train, np.ravel(y_train))
y_hat_train = logreg.predict(X_train)
y_hat_test = logreg.predict(X_test)

In [94]:
train_mse = mean_squared_error(y_train, y_hat_train, squared = False)
test_mse = mean_squared_error(y_test, y_hat_test, squared = False)

In [95]:
train_mse

0.4312736994803176

In [96]:
test_mse

0.43781852695607765